# Bank Statement Analysis 

Jupyter Notebook created to categorize .csv based bank statment expances.  

Currently suppoorted bank account export format is `ING Bank Śląski` which can aggreagate expances form other banks.

# Export data from Bank accounts


**ING Bank Śląski** -  Export the accounts you would like to do analysis for. Usually I'm esporting expances from all accounts except `Rachunki` bank account to see how much money was spenf to things like grocery. Exported file have `.csv` format and `cp-1250` encoding.

Data presnetation article
https://www.shanelynn.ie/data-visualisation-in-python-pycon-dublin-2018-presentation/




## Input Data

In [34]:
ING_CSV = "data/ing_mil_24_25.csv"

## Categorization

In [35]:
CATEGORY = {}

# 1. ING Bank Śląski



## 1.1 Import Data from ING Bank Sląski

Import raw data from .csv file.

`header` - is an index of non empty row in input csv file. Will be used as `df` column name.


In [36]:
import pandas as pd
from tabulate import tabulate

from helper import get_ing_header_row_index

# Dataframe print settings
pd.set_option('display.max_rows', None) # Show all rows
pd.set_option('display.max_columns', None) # Show all columns
pd.set_option('display.max_colwidth', None) # Optional: prevent truncating column width
pd.set_option('display.width', None) # Optional: wide display for Jupyter environments

# CSV file settings 
ING_ENCODING = "cp1250"
ING_SEP = ";"

ing_raw_df = pd.read_csv(
    ING_CSV,
    encoding=ING_ENCODING,
    sep=ING_SEP,
    header=get_ing_header_row_index(
        ING_CSV, ING_ENCODING
    ),
)

# Trim data frame - Drop last row which is empty
ing_raw_df.drop(ing_raw_df.tail(1).index, inplace=True)

## 1.2 - Data preparation

Ceanup the data from ING Bank account to extract only usefull columns

* Trim data fame to get only usefull columns for analysis
* Manipulate data
* Set data types


In [ ]:
# *** TRIM DATA FRAME ***

# Drop columns with "Unnamed" in the header
df = ing_raw_df.loc[:, ~ing_raw_df.columns.str.startswith("Unnamed")]


# *** MODIFY DATA FRAME *** 

"""
COPY Kwota blokady/zwolnienie blokady ->  Kwota transakcji (waluta rachunku). 
This is the case when the transaction is a blocked.
"""

df = df.copy() # needed due to error
df.loc[
    df["Kwota transakcji (waluta rachunku)"].isna(),
    "Kwota transakcji (waluta rachunku)"
] = df["Kwota blokady/zwolnienie blokady"]

"""
RENAME Kwota transakcji (waluta rachunku) -> Kwota transakcji
"""

df = df.rename(columns={"Kwota transakcji (waluta rachunku)": "Kwota transakcji"})


"""
IF Dane kontrahenta empty COPY Tytuł
"""

df["Dane kontrahenta"] = df.apply(
    lambda row: row["Tytuł"] if not row["Dane kontrahenta"].strip() else row["Dane kontrahenta"],
    axis=1
)

# *** Select only needed columns ***

"""
Columns: Data transakcji; Dane kontrahenta; Kwota transakcji; Kategoria
"""
df = df.copy()
df = df[["Data transakcji", "Dane kontrahenta", "Kwota transakcji"]]
df["Kategoria"] = "NaN"


# *** SET data types for columns ***

df["Data transakcji"] = df["Data transakcji"].astype("datetime64[ns]")
df["Dane kontrahenta"] = df["Dane kontrahenta"].astype(str)

df["Kwota transakcji"] = df["Kwota transakcji"].str.replace(",", ".").astype(float) # Replace comma with dot
df["Kwota transakcji"] = df["Kwota transakcji"].astype(float)
df["Kwota transakcji"] = -df["Kwota transakcji"] # Change the sign of "Kwota transakcji" column

df["Kategoria"] = df["Kategoria"].astype(str)


In [38]:
df.dtypes

Data transakcji     datetime64[ns]
Dane kontrahenta            object
Kwota transakcji           float64
Kategoria                   object
dtype: object

In [39]:
df.shape

(2892, 4)

In [40]:
df.tail(10)

,Data transakcji,Dane kontrahenta,Kwota transakcji,Kategoria
2882,2024-01-02,ZABKA ZB312 K.2 DOBRZYKOW,23.79,NaN
2883,2024-01-02,ZABKA ZB312 K.2 DOBRZYKOW,11.69,NaN
2884,2024-01-02,PIZZERIA TUTTI SANTI WROCLAW,117.98,NaN
2885,2024-01-01,PIAZZA PIZZERIA Wroclaw POL,69.00,NaN
2886,2024-01-01,Myjnia NIKITA Sz. Klacz Wroclaw P,6.00,NaN
2887,2024-01-01,BP-KRASNAL 386 MOKRONOS DOLN POL,13.97,NaN
2888,2024-01-01,PlayStation Network Hilversum,244.30,NaN
2889,2024-01-01,"PIEKARSTWO"" LANY """,20.90,NaN
2890,2024-01-01,ETNO CAFE WROCLAW,30.80,NaN
2891,2024-01-01,SUPER-PHARM WROCLAW MAG WROCLAW,23.99,NaN


## 1.3 Categorize ING Bank Statment Data


In [41]:
# Keywords for each category
category_keywords = {
    "Zakupy spożywcze": ["biedronka", "lidl", "kaufland", "carrefour", "netto", "zabka", "freshmarket",
                          "stokrotka", "auchan", "selgros", "tesco", "intermarche", "leclerc", "freshmarket", "piekar"],
    # "Paliwo": ["orlen", "bp", "shell"],
    # "Restauracje": ["mcdonald", "kfc", "pizza"],
}

# Lowercase "Dane kontrahenta" for case-insensitive matching
df["Dane_lower"] = df["Dane kontrahenta"].str.lower()


# Function to find category
def find_category(dane):
    for category, keywords in category_keywords.items():
        if any(keyword in dane for keyword in keywords):
            return category
    return "Inne"  # default if no match

# Apply the function
df["Kategoria"] = df["Dane_lower"].apply(find_category)

# Drop helper column if not needed
df.drop(columns=["Dane_lower"], inplace=True)


# DEBUG: Save the dataframe as an Excel file
# df.to_csv("test.csv", sep=";", index=False, encoding=ING_ENCODING)

df_categorized =  df[df["Kategoria"] == "Zakupy spożywcze"]


## 1.4 Data Visualisation


/var/folders/2p/vjq15lqs1c7c9m_kcl6hj4d00000gn/T/ipykernel_67962/2322985660.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_categorized["Month"] = df_categorized["Data transakcji"].dt.to_period("M")


In [ ]:
import matplotlib.pyplot as plt

# Group by month and sum the transaction amounts for "Zakupy spożywcze"
df_categorized["Month"] = df_categorized["Data transakcji"].dt.to_period("M")
monthly_summary = df_categorized.groupby("Month")["Kwota transakcji"].sum()

# Calculate the average value
average_value = monthly_summary.mean()

# Plot the monthly summary
ax = monthly_summary.plot(kind="bar", figsize=(10, 6), color="skyblue", label="Monthly Total")
plt.axhline(y=average_value, color="red", linestyle="--", label=f"Average ({average_value:.2f})")
plt.title("Zakupy spożywcze - Monthly Summary")
plt.xlabel("Month")
plt.ylabel("Total Amount (PLN)")
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)

# Add data labels at the end of each bar
for i, value in enumerate(monthly_summary):
    ax.text(i, value + 50, f"{value:.2f}", ha="center", va="bottom", fontsize=9)

plt.legend()
plt.tight_layout()
plt.show()
